In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/multilabel-3000-10epochs/multi-label-binarizer.pkl
/kaggle/input/multilabelbinarizer-3000-15epochs-cross/multi-label-binarizer.pkl
/kaggle/input/encodedcsv-3000-15epochs-cross/encodedCSV.csv
/kaggle/input/datasetcsv54k-12epochs-cross/encodedCSV.csv
/kaggle/input/cleanedproductdata81k8batch/newData81k.csv
/kaggle/input/model-3000-15epochs-cross/config.json
/kaggle/input/model-3000-15epochs-cross/training_args.bin
/kaggle/input/model-3000-15epochs-cross/model.safetensors
/kaggle/input/model-3000-10epochs/config.json
/kaggle/input/model-3000-10epochs/training_args.bin
/kaggle/input/model-3000-10epochs/model.safetensors
/kaggle/input/tokenizer-3000-10epochs/tokenizer_config.json
/kaggle/input/tokenizer-3000-10epochs/special_tokens_map.json
/kaggle/input/tokenizer-3000-10epochs/vocab.txt
/kaggle/input/tokenizer-54k-12epoch-cross/tokenizer_config.json
/kaggle/input/tokenizer-54k-12epoch-cross/special_tokens_map.json
/kaggle/input/tokenizer-54k-12epoch-cross/vocab.txt
/kaggle/in

# **Jai Hanuman Dada ki Jai**

In [ ]:
# df_new=pd.read_csv('/kaggle/input/preprocesseddata1lac/Testtfidftransformer1Lac.csv')
df=pd.read_csv('/kaggle/input/cleanedproductdata81k8batch/newData81k.csv')

In [ ]:
# Create an empty DataFrame to store the sampled data
sampled_data = pd.DataFrame()

# Group the DataFrame by 'main_category'
grouped = df.groupby('main_category')

# Iterate over each group and sample 2000 rows
for group_name, group_data in grouped:
    sampled_data = pd.concat([sampled_data, group_data.sample(n=2000, random_state=42)])
    
df = sampled_data

In [ ]:
df = df.sample(54000)

In [ ]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

#Download NLTK resources
import nltk
import subprocess
nltk.download('punkt')
nltk.download('stopwords')

try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')
    
# Function for text preprocessing
def preprocess_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [ ]:
df['CummProdInfo'] = df['CummProdInfo'].apply(preprocess_text)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['title'])

# Applying TFIDF to find 50 similar items to each item.

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

# # TF-IDF Vectorization
# tfidf_vectorizer = TfidfVectorizer()
# tfidf_matrix = tfidf_vectorizer.fit_transform(df['CummProdInfo'])

# # Calculate cosine similarity between each row and all other rows
# cosine_similarities = cosine_similarity(tfidf_matrix)

# # Find the indices of the top 50 most similar items for each row
# top_50_indices = cosine_similarities.argsort(axis=1)[:, :-51:-1]

# # Get the label values from the original DataFrame that match the top 50 most similar items for each row
# top_50_labels = [[df['label'].iloc[i] for i in indices] for indices in top_50_indices]

# # Add the list of labels as values of each row into a new column called label_list
# df['label_list'] = top_50_labels

# # Display the DataFrame
# df


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import numpy as np

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['CummProdInfo'])

# Define a function to calculate cosine similarity for a chunk of data
def calculate_cosine_similarity(chunk):
    return cosine_similarity(chunk, tfidf_matrix)

# Initialize an empty dictionary to store the top 50 most similar items for each row
top_50_labels = {}

# Process the data in chunks
chunk_size = 1000
num_chunks = int(np.ceil(tfidf_matrix.shape[0] / chunk_size))
for chunk_index in range(num_chunks):
    start_index = chunk_index * chunk_size
    end_index = min((chunk_index + 1) * chunk_size, tfidf_matrix.shape[0])
    chunk = tfidf_matrix[start_index:end_index]

    # Calculate cosine similarity for the chunk
    cosine_similarities_chunk = calculate_cosine_similarity(chunk)
    
    # Find the indices of the top 50 most similar items for each row in the chunk
    for i in range(len(cosine_similarities_chunk)):
        top_50 = heapq.nlargest(50, range(len(cosine_similarities_chunk[i])), key=lambda k: cosine_similarities_chunk[i][k])
        top_50_labels[start_index + i] = [df['label'].iloc[k] for k in top_50]

# Add the list of labels as values of each row into a new column called label_list
df['label_list'] = [top_50_labels.get(i, []) for i in range(len(df))]

# Display the DataFrame
df



In [ ]:
print("TFIDF done")

In [ ]:
df.info()

In [ ]:
# Get the unique titles in the 'title' column
unique_title = df['main_category'].unique()
print(f"The unique titles in the 'tile' column are: {unique_title}")

# Get the count of each tile in the 'title' column
category_counts = df['main_category'].value_counts()
print("\nThe count of each title is:")
print(category_counts)

In [ ]:
df['CummProdInfo'].str.len().plot.hist(bins=40)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
multilabel = MultiLabelBinarizer(sparse_output=True)

labelTemp = df['label_list'].tolist()

labels = multilabel.fit_transform(labelTemp)
labels = labels.astype('float32')

labels = labels.toarray()

texts = df['CummProdInfo'].tolist()

In [ ]:
labels

In [ ]:
# labels
# np.set_printoptions(threshold=np.inf)

In [ ]:
import os
import torch
from transformers import DistilBertTokenizer, AutoTokenizer
from transformers import DistilBertForSequenceClassification, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Split data into train and validation sets
train_inputs, val_inputs, train_labels, val_labels = train_test_split(texts, labels, test_size=0.15, random_state=42)

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = len(labels[0]),
                                                           problem_type="multi_label_classification")

In [ ]:
class ECommerceFoundDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_token_len=512):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        prodInfo = str(self.texts[index])
        label = torch.tensor(self.labels[index])

        inputs = self.tokenizer(
            prodInfo,
            add_special_tokens=True,
            padding="max_length",
            truncation=True,
            max_length=self.max_token_len,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': label
        }

In [ ]:
train_dataset = ECommerceFoundDataset(train_inputs, train_labels, tokenizer)
val_dataset = ECommerceFoundDataset(val_inputs, val_labels, tokenizer)

In [ ]:
#Evaluation Metrics
from sklearn.metrics import roc_auc_score, f1_score, hamming_loss
from transformers import EvalPrediction

def compute_metrics_multilabel(predictions, labels, threshold=0.5):
    sigmoid = torch.nn.Sigmoid()
    probability = sigmoid(torch.Tensor(predictions))
    y_pred = np.zeros(probability.shape)
    y_pred[np.where(probability>=threshold)] = 1
    y_true_value = labels
    f1_scoreV = f1_score(y_true_value, y_pred, average='macro')
    
    # Check if there is only one class present in y_true
    if len(np.unique(y_true_value)) == 1:
        # Handle the edge case where there is only one class
        roc_auc = None
    else:
        # Calculate ROC AUC only if there are multiple classes
        try:
            roc_auc = roc_auc_score(y_true_value, y_pred, average='macro')
        except ValueError:
            # Handle the case where roc_auc_score throws an error due to single class
            roc_auc = None
        
    hamming = hamming_loss(y_true_value, y_pred)
    
    metrics_value = {
        "f1_score":f1_scoreV,
        "hamming_loss":hamming,
        "roc_auc": roc_auc
    }
    
    return metrics_value

def compute_metrics(p:EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = compute_metrics_multilabel(predictions=preds,labels=p.label_ids)
    
    return result

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=5e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    output_dir='./results',
    num_train_epochs=12,
    save_steps=1000,
    save_total_limit=2
)

trainer = Trainer(model= model,
                 args=args,
                 train_dataset=train_dataset,
                 eval_dataset=val_dataset,
                 data_collator=data_collator,
                 compute_metrics=compute_metrics)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

# Validation

In [ ]:
# text = "I want a soft pillow" #Good
# text = "baby"  #Good
# text = "music"  #Good
# text = "pillow"  #Good
# text = "diaper"  #Bad
# text = "i want beauty items" #Good
# text = "teeth whitening item" #Bad
# text = "I want shoes" #Good
# text = "I want to buy an acoustic guitar" #Good
# text = "I want to buy a camera" #Good
# text = "I want to buy pen camera" #Good
# text = "Suggest me baby products, handmade and camera" #Good
# text = "Suggest me baby products and guitar"#Good
text = "I want to buy a diaper" #Good



text = preprocess_text(text)
text

encoding = tokenizer(text, return_tensors='pt')
encoding.to(model.device)

outputs = model(**encoding)


import torch
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0].cpu())

# Convert probabilities to numpy array
probs_np = probs.detach().numpy()

# Thresholding to get predictions
preds = np.zeros(probs_np.shape)

preds[np.where(probs_np >= 0)] = 1

# Inverse transform to get labels
values = multilabel.inverse_transform(preds.reshape(1, -1))

values = np.array(values[0])

indices = np.where(preds == 1)

# Extract the corresponding probability values for these indices
label_probabilities = probs_np[indices]

# Sort label_probabilities array in descending order and retain indices
sorted_indices = np.argsort(label_probabilities)[::-1]
sorted_label_probabilities = label_probabilities[sorted_indices]

# Extract values from the list 'values' based on the sorted indices
sorted_values = values[sorted_indices]

print(sorted_label_probabilities)
print(sorted_values)


# Reset index of the DataFrame
df_reset_index = df.reset_index(drop=True)

# Create a categorical variable with custom ordering
custom_order = pd.Categorical(df_reset_index['label'], categories=sorted_values, ordered=True)

# Sort the DataFrame by the custom ordering
df_sorted = df_reset_index.loc[custom_order.argsort()]

# Select rows where label is in sorted_values
selected_rows = df_sorted[df_sorted['label'].isin(sorted_values)]

selected_rows[:40]

In [ ]:
trainer.save_model("distilbert-finetuned-ecommerce")
tokenizer.save_pretrained("/kaggle/working/tokenizerRS")
df.to_csv("encodedCSV.csv",index=False)
import pickle
with open("multi-label-binarizer.pkl","wb") as f:
    pickle.dump(multilabel,f)

# Loading Stored model, tokenizer, labelled_csv, multilabelBinarizer

# Testing

In [2]:
import os
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import pickle


# Load the tokenizer
tokenizer1 = DistilBertTokenizer.from_pretrained("/kaggle/input/tokenizer-54k-12epoch-cross")

# Load the model
model1 = DistilBertForSequenceClassification.from_pretrained("/kaggle/input/model-54k-12epochs-cross")

#load the label encoded csv
df1 = pd.read_csv('/kaggle/input/datasetcsv54k-12epochs-cross/encodedCSV.csv')

# Load the multi-label binarizer
with open("/kaggle/input/multilabel-binarizer-54k-12epochs-cross/multi-label-binarizer.pkl", "rb") as f:
    multilabel1 = pickle.load(f)

In [3]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

#Download NLTK resources
import nltk
import subprocess
nltk.download('punkt')
nltk.download('stopwords')

try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')
    
# Function for text preprocessing
def preprocess_text(text):
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into a string
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text




[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /kaggle/working/...
Archive:  /kaggle/working/corpora/wordnet.zip
   creating: /kaggle/working/corpora/wordnet/
  inflating: /kaggle/working/corpora/wordnet/lexnames  
  inflating: /kaggle/working/corpora/wordnet/data.verb  
  inflating: /kaggle/working/corpora/wordnet/index.adv  
  inflating: /kaggle/working/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/corpora/wordnet/index.verb  
  inflating: /kaggle/working/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/corpora/wordnet/data.adj  
  inflating: /kaggle/working/corpora/wordnet/index.adj  
  inflating: /kaggle/working/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/corpor

In [5]:
# text = "I want a soft pillow" #Good
# text = "baby"  #Good
# text = "suggest me good music"  #Good
# text = "pillow"  #Good
# text = "diaper"  #Bad
# text = "i want beauty items" #Good
# text = "teeth whitening item" #Bad
# text = "grocery items" #Good
# text = "suggest me some artwork" #Good
# text = "I want shoes" #Good
# text = "I need tea" #Good
# text = "suggest me baby guitar"#Good
# text = "I want cameras and amazon fashion"#Good
# text = "I want sweater" #Even though sweater not present in the dataset , it belongs to the 
                        #category amazon fashion
# text = "I want mobile phone" #good
# text = "I want fridge" #good(here referigerator is there...still able to find fridge )
# text = "Suggest me chocolates" #good
text = "I have an birthday party tommorow. Suggest me some products"#Good
# text="I want coffee and tea" #Good
# text = "Want a very fast computational device " #it is understanding that I need a fast computational device 
# text = "Want a bottle and shoes"  #understanding both of the concept
# text = "Want a gas and cooker"

text = preprocess_text(text)
text

encoding = tokenizer1(text, return_tensors='pt')
encoding.to(model1.device)

outputs = model1(**encoding)



import torch
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(outputs.logits[0].cpu())

# Convert probabilities to numpy array
probs_np = probs.detach().numpy()

# Thresholding to get predictions
preds = np.zeros(probs_np.shape)

preds[np.where(probs_np >= 0)] = 1

# Inverse transform to get labels
values = multilabel1.inverse_transform(preds.reshape(1, -1))

values = np.array(values[0])

indices = np.where(preds == 1)

# Extract the corresponding probability values for these indices
label_probabilities = probs_np[indices]

# Sort label_probabilities array in descending order and retain indices
sorted_indices = np.argsort(label_probabilities)[::-1]
sorted_label_probabilities = label_probabilities[sorted_indices]

# Extract values from the list 'values' based on the sorted indices
sorted_values = values[sorted_indices]

# print(sorted_label_probabilities)
# print(sorted_values)




# Reset index of the DataFrame
df_reset_index = df1.reset_index(drop=True)

# Create a categorical variable with custom ordering
custom_order = pd.Categorical(df_reset_index['label'], categories=sorted_values, ordered=True)

# Sort the DataFrame by the custom ordering
df_sorted = df_reset_index.loc[custom_order.argsort()]

# Select rows where label is in sorted_values
selected_rows = df_sorted[df_sorted['label'].isin(sorted_values)]

selected_rows[:60]

,main_category,title,CummProdInfo,average_rating,label,label_list
23958,Camera & Photo,Gatherfun Birthday Party Supplies Happy 25th B...,product gatherfun birthday party supply happy ...,4.8,19766,"[19766, 9813, 47353, 21961, 36272, 21951, 2717..."
28789,Toys & Games,"Crseniny Happy Birthday Cake Topper,Gold Glitt...",product crseniny happy birthday cake topper go...,4.7,12640,"[12640, 21949, 29749, 3942, 35381, 4020, 48075..."
18252,Toys & Games,Level 9 Game Birthday Cake Topper - Video Game...,product level game birthday cake topper video ...,4.7,27622,"[27622, 12640, 11275, 1399, 35381, 194, 29749,..."
41123,Handmade,42nd Birthday Party Decorations for 42nd Birth...,product nd birthday party decoration nd birthd...,4.7,2102,"[2102, 16926, 20437, 19766, 12640, 47353, 3101..."
36844,Camera & Photo,Pimvimcim 17th Birthday Banner Decorations for...,product pimvimcim th birthday banner decoratio...,4.4,36272,"[36272, 47353, 2095, 19766, 21951, 25572, 2198..."
35670,Camera & Photo,Mocsicka 3rd Donut Birthday Backdrop Three is ...,product mocsicka rd donut birthday backdrop th...,4.7,31013,"[31013, 31017, 6076, 9813, 19766, 31018, 21989..."
4434,Toys & Games,Party Decorations Supplies cartoon game us The...,product party decoration supply cartoon game u...,4.3,35381,"[35381, 1719, 9964, 12640, 8053, 1381, 9435, 9..."
9279,Camera & Photo,Large Spiderman Happy Birthday Banner | Superh...,product large spiderman happy birthday banner ...,4.6,27179,"[27179, 19766, 9813, 31013, 21961, 47353, 3444..."
34012,Toys & Games,213pcs Video Gamer Birthday Party Decorations ...,product pc video gamer birthday party decorati...,4.4,1259,"[1259, 28508, 35381, 1381, 677, 19768, 47354, ..."
24684,Toys & Games,Dinosaur Party - Happy Birthday Banner | Volca...,product dinosaur party happy birthday banner v...,4.5,14322,"[14322, 3579, 21439, 31012, 34861, 19766, 1432..."
